# Post Earthquake Satellite Autotasking Project
This Jupyter Notebook will contain all code related to the accessing and tasking of satellites following an earthquake. The goal is to automate the process of image capture tasks for earthquake-affected areas using a satellite tasking API. Please replace 'SATELLITE' with your actual satellite tasking API provider for specific instructions.

## Example Usage
Example code showing how to access the SATELLITE API, capture images, and manage tasks will go here.